In [1]:
import numpy as np
import pandas as pd


# read in roi data and parse into regs and bes (breakevens) 
roi = pd.read_csv('roi.csv')
regs = roi[roi.reg == 1]
bes = roi[roi.be == 1]
ban_from_study = ['MysticRiver5', 'unclepete55','cu4rp3rb3r','Aethyr13', 'reTIRED49']

# reg_ulate takes a dataframe and creates ptype columns and notates reg and breakeven players
def id_ptype(df):
    df['ptype'] = 'rec'
    for name in regs.name:
        df.loc[df.Player == name, 'ptype'] = 'reg'
        
    for name in bes.name:
        df.loc[df.Player == name, 'ptype'] = 'be'

    return df
# read in hh using list comprehension
hh = [line.rstrip('\n') for line in open('hh.txt')]
# display hh first 5 rows
hh[:5]

['Winning Poker Network Game #1081326940: No Limit Holdem (25/50) [2017/12/22 17:59:45 UTC]',
 'Table: $10 Regular 9-Max, Table 1',
 'Tournament: 8140214',
 'Seats: 9',
 'Seat 1: W1renut (1,439)']

In [2]:
def get_stacks():
    # line.split()[1] is a string that i slice the first char with [:1]
    seats = [line.split()[1][:1] for line in hh if '(' in line]
    # convert seats to int and drop first and last items in seats list
    seats = [int(seat) for seat in seats[1:-1]]
    
    # get names associated with each seat
    names = [line.split()[2:-1] for line in hh if '(' in line]
    names = [name for name in names[1:-1]]
    
    # get stacks associated with each seat
    stacks = [line.split()[-1] for line in hh if '(' in line]
    # drop parenthesis on each stack and drop the comma and convert stack to int
    # note: dropping first and last items in list as in seats
    stacks = [int(stack[1:-1].replace(',', '')) for stack in stacks[1:-1]]
    
    # identify which seat is the button
    # note this list comprehension only returns a one element list which I extract
    btn = [int(line.split()[-1]) for line in hh if 'Button is seat ' in line][0]
    
    index_btn = seats.index(btn)
    
    # create position list in appropriate deal order for 9 players PT4 convention
    pos = [6,5,4,3,2,1,0,9,8]
    pos = pos[9-len(stacks):]
    num_stacks = len(stacks)
    # repeat list to create a window
    stacks1 = (stacks * 2)[index_btn:index_btn+num_stacks]
    names1 = (names * 2)[index_btn:index_btn+num_stacks]
    # join stacks to match deal order and positions
    stacks2 = stacks1[3:] + stacks1[:3]
    names2 = names1[3:] + names1[:3]
    df = pd.DataFrame({'pos': pos, 'name': names2, 'chips': stacks2}).set_index('pos')
    
    return df

get_stacks()


,name,chips
pos,,
5,[Truck1536],3141
4,[allegro88],1670
3,[unclepete55],1721
2,[Cutch123],1310
1,[cobblestoner],1201
0,[otlichno],1215
9,"[O.G., SKARFINGER]",1803
8,[W1renut],1439


In [3]:
def get_bb():
    bb = [int(line.split()[-1]) for line in hh if 'posts big blind' in line][0]
    return bb

get_bb()

50

In [4]:
def get_sb():
    sb = [int(line.split()[-1]) for line in hh if 'posts small blind' in line][0]
    return sb

get_sb()

25

In [3]:
# for the purpose of my study I am calculating 9-man sng payout structure
# 0.5, 0.3, 0.2 to 1st, 2nd, 3rd respectively
def ICM9(df):
    payouts = [0.5, 0.3, 0.2]
    tot = df.chips.sum()
    length = len(df.chips)
    # df.left1 is the amount of chips remaining if that player wins 1st place
    df['left1'] = tot - df.chips
    
    df['p1st'] = df.chips / tot
    
    
    # left2 is a df of chips remaining if players i and j get 1st and 2nd
    # note that who gets first and who gets second is irrelevant since the chips remaining
    #      will be the same
    left2 = pd.DataFrame(np.nan, index = range(length), columns = range(len(df.chips)))
    for i in range(len(df.chips)-1):
        for j in range(i+1,len(df.chips)):
            left2.iloc[i,j] = tot - df.chips.iloc[i] - df.chips.iloc[j]
    
    
    p2 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
    # i is the row and represents the player placing first in the tournament
    for i in range(length):
        # p2[i,j] is the jth player's equity in 2nd place given player i is the winner
        for j in range(length):
            # if i == j then player cannot be second because she has already won
            if i != j:
                p2.iloc[i,j] = df.p1st.iloc[i] * df.chips.iloc[j] / df.left1.iloc[i] 
                
    df['eq1st'] = df.p1st * payouts[0]
    p2nd = [p2[i].sum() for i in range(length)]
    df['p2nd'] = p2nd
    df['eq2nd'] = df.p2nd * payouts[1]
    
    # multiply p2 and chips[i] / left2
    
    p3_list = []
    for ind, stack in enumerate(df.chips):
        p3 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
        for i in range(length):
            for j in range(i+1, length):
                if (i != ind) & (j != ind):
                    p3.iloc[i,j] = stack / left2.iloc[i,j] * p2.iloc[i,j]
                    p3.iloc[j,i] = stack / left2.iloc[i,j] * p2.iloc[j,i]
        
        p3_list.append(p3.sum().sum())
    
    df['p3rd'] = p3_list
    df['eq3rd'] = df.p3rd * payouts[2]
    df['$EV'] = df.eq1st + df.eq2nd + df.eq3rd 
    df['pITM'] = df.p1st + df.p2nd + df.p3rd
    return df[['name', 'chips','$EV', 'pITM']]

res = ICM9(get_stacks())
res




,name,chips,$EV,pITM
pos,,,,
5,[Truck1536],3141,0.210669,0.603748
4,[allegro88],1670,0.125891,0.380391
3,[unclepete55],1721,0.129258,0.389933
2,[Cutch123],1310,0.101268,0.309329
1,[cobblestoner],1201,0.093522,0.286556
0,[otlichno],1215,0.094524,0.289513
9,"[O.G., SKARFINGER]",1803,0.134606,0.404998
8,[W1renut],1439,0.110262,0.335532


In [74]:
rfinai = pd.read_csv('rfinai.csv')
id_ptype(rfinai)

unopened = pd.read_csv('unopened.csv')
id_ptype(unopened)

lfi = pd.read_csv('lfi.csv')
id_ptype(lfi)

os = pd.read_csv('os.csv')
id_ptype(os)

# Purpose: calculate portions and 90% confidence intervals
def calc_preflop_portions(rfinai, limp, allin, unopened):
    rfi = rfinai.iloc[:, 3:-1].sum()
    lfi = limp.iloc[:, 3:-1].sum()
    os = allin.iloc[:, 3:-1].sum()
    unop = unopened.iloc[:, 3:-1].sum()
    datas = [rfi, lfi, os, unop]
    ind = ['0-4','4-8','0-8', '8-11','11-14','14-20','20+']
    cols = ['rfi','lfi','os','unopen']
    for data in datas:
        data.index = ind
    
    df = pd.DataFrame({'rfi': rfi, 'lfi': lfi, 'os': os, 'unopen': unop}, index=ind, columns=cols)
    df['prfi'] = df.rfi / df.unopen
    df['plfi'] = df.lfi / df.unopen
    df['pos'] = df.os / df.unopen
    df['pother'] = 1 - df.prfi - df.plfi - df.pos
    
    return df
calc_preflop_portions(rfinai,lfi, os, unopened)

    

    
    

,rfi,lfi,os,unopen,prfi,plfi,pos,pother
0-4,2545,1306,6602,15072,0.168856,0.086651,0.438031,0.306462
4-8,1913,3028,24277,70239,0.027236,0.043110,0.345634,0.584020
0-8,4455,4330,30820,85149,0.052320,0.050852,0.361954,0.534874
8-11,4246,5676,18980,82806,0.051276,0.068546,0.229210,0.650967
11-14,8635,7494,11052,85947,0.100469,0.087193,0.128591,0.683747
14-20,18156,11800,5372,115822,0.156758,0.101880,0.046382,0.694980
20+,47727,36064,1182,358518,0.133123,0.100592,0.003297,0.762988
